In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pydataset import data
import seaborn as sns
# acquire
from env import user, password, hostname, get_db_url
from pydataset import data
from sklearn.model_selection import train_test_split


import acquire
import prepare

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

SyntaxError: invalid syntax (prepare.py, line 1)

In [ ]:
df_iris = data('iris', show_doc=True)

In [ ]:
df_iris = data('iris')
df_iris.head(3)

In [ ]:
df_iris.shape

In [ ]:
df_iris.columns

In [ ]:
df_iris.dtypes

In [ ]:
df_iris.info()

In [ ]:
stats = df_iris.describe().T
stats

## ACQUIRE Google Sheets

In [ ]:
# Grab the Google sheet url.

sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'

In [ ]:
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [ ]:
df_google = pd.read_csv(csv_export_url)

In [ ]:
df_google.info

In [ ]:
df_google.head(3)

In [ ]:
df_google.shape

In [ ]:

# shows columns and puts them into list more functionality 
df_google.columns.to_list()

In [ ]:
df_google.dtypes


In [ ]:
df_google.describe().T


In [ ]:
for col in df_google.columns:
#     print(col)
    if df_google[col].dtypes == 'object':
        print(f'{col} has {df_google[col].nunique()} unique values.')

In [ ]:
df_google.Survived.value_counts(dropna=False)

In [ ]:
df_google.Pclass.value_counts(dropna=False)

In [ ]:
df_google.Embarked.value_counts(dropna=False)

## ACQUIRE EXCEL

Read the train table from the train.xlsx file into a dataframe named df_excel.

In [ ]:
df_excel = pd.read_excel('train.xlsx', sheet_name ='train')

In [ ]:
df_excel.info()

In [ ]:
df_excel_sample = df_excel.head(100)

In [ ]:
df_excel_sample.shape

In [ ]:
df_excel.shape[0]

In [ ]:
df_excel.columns[:5]

In [ ]:
df_excel.select_dtypes(include='object').head()

In [ ]:
df_excel.select_dtypes(include='object').columns.to_list()


In [ ]:
df_excel.head(3)

In [ ]:
titanic_stats = df_excel[['Age', 'Fare']].describe().T
titanic_stats

## CREATE acquire.py

In [ ]:
import acquire

In [ ]:
titanic_df = acquire.get_titanic_data()

In [ ]:
titanic_df.head()

In [ ]:
iris_df = acquire.get_iris_data()


In [ ]:
iris_df.head()


In [ ]:
telco_df = acquire.get_telco_data()

In [ ]:
telco_df.head()

## Exploratory Analysis

### Part 1
Continue in your classification_exercises.ipynb notebook. As always, add, commit, and push your changes.

Section 1 - iris_db: Using iris data from our mySQL server and the methods used in the lesson above:

#### 1. Acquire, prepare & split your data.

In [ ]:
# get iris data
df_iris = acquire.get_iris_data()
df_iris

In [ ]:
df_iris.shape

In [ ]:
 #Drop the unnecessary column
df_iris = df_iris.drop(columns=['Unnamed: 0', 'measurement_id'])
df_iris


In [ ]:
df_iris = df_iris.rename(columns={'species_name':'species', 'species_id': "id"})
df_iris

In [ ]:
def my_train_test_split(df, target = None):
    if target:
        train, test = train_test_split(df, test_size=.2, random_state=123, stratify=df[target])
        train, validate = train_test_split(train, test_size=.25, random_state=123, stratify=train[target])
    else:
        train, test = train_test_split(df, test_size=.2, random_state=123)
        train, validate = train_test_split(train, test_size=.25, random_state=123)
    return train, validate, test

In [ ]:
target = 'species'

In [ ]:
train_iris, validate_iris, test_iris = my_train_test_split(df_iris, target = 'species')

In [ ]:
print(f'Prepared df: {df_iris.shape}')
print()
print(f'Train: {train_iris.shape}')
print(f'Validate: {validate_iris.shape}')
print(f'Test: {test_iris.shape}')

#### 2. Univariate Stats

For each measurement type (quantitative variable): create a histogram, boxplot, & compute descriptive statistics (using .describe()).

In [ ]:
iris_quant_values = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']



for variable in iris_quant_values:
    train_iris[variable].hist()
    plt.title(variable)
    plt.ylabel('Frequency')
    plt.xlabel('Value')
    plt.show()
    print(train_iris[variable].describe())
    print()



In [ ]:
iris_quant_values = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

for variable in iris_quant_values:
    sns.boxplot(x=variable, data=train_iris)
    plt.title(variable)
    plt.show()
    print(train_iris[variable].describe())
    print()

   

In [ ]:
 # Compute descriptive statistics

iris_stats = df_iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].describe()
iris_stats

For each species (categorical variable): create a frequency table and a bar plot of those frequencies.

In [ ]:
categorical_vars = ['species']

# Frequency table
iris_freq = df_iris['species'].value_counts()

for col in categorical_vars:
    print(train_iris[col].value_counts(normalize=True) * 100)
    sns.countplot(x=col, data=train_iris)
    plt.show()


Document takeaways & any actions.

#### 3. Bivariate Stats

Visualize each measurement type (y-axis) with the species variable (x-axis) using barplots, adding a horizontal line showing the overall mean of the metric (y-axis).

For each measurement type, compute the descriptive statistics for each species.

For virginica & versicolor: Compare the mean petal_width using the Mann-Whitney test (scipy.stats.mannwhitneyu) to see if there is a significant difference between the two groups. Do the same for the other measurement types.

Document takeaways & any actions.

In [ ]:
# Frequency table
species_freq = df_iris['sepal_length'].value_counts()


plt.figure(figsize=(8, 6))
plt.bar(df_iris['species'].unique(), df_iris.groupby('species')['sepal_length'].mean(), color='pink', linestyle='--')
plt.xlabel('Species')
plt.ylabel('Sepal Length')
plt.title('Average Sepal Length by Species')
plt.legend(['Overall Mean'])
plt.show()

In [ ]:
# # Frequency table
# species_freq = df_iris['sepal_width'].value_counts()

# Bar plot
plt.figure(figsize=(8, 6))
plt.bar(df_iris['species'].unique(), df_iris.groupby('species')['sepal_width'].mean(), color='pink', linestyle='--')
plt.xlabel('Species')
plt.ylabel('sepal_width')
plt.title('Average Sepal Wodth by Species')
plt.legend(['Overall Mean'])
plt.show()




In [ ]:
# Bar plot
# Bar plot
plt.figure(figsize=(8, 6))
plt.bar(df_iris['species'].unique(), df_iris.groupby('species')['petal_length'].mean(), color='pink', linestyle='--')
plt.xlabel('Species')
plt.ylabel('petal_length')
plt.title('Average Petal Length by Species')
plt.legend(['Overall Mean'])
plt.show()

In [ ]:
# Bar plot
# Bar plot
plt.figure(figsize=(8, 6))
plt.bar(df_iris['species'].unique(), df_iris.groupby('species')['petal_width'].mean(), color='pink', linestyle='--')
plt.xlabel('Species')
plt.ylabel('sepal_width')
plt.title('Average Petal Width by Species')
plt.legend(['Overall Mean'])
plt.show()

#### 4. Multivariate Stats

Visualize the interaction of each measurement type with the others using a pairplot (or scatter matrix or something similar) and add color to represent species.

Visualize two numeric variables of the species. Hint: sns.relplot with hue or col

Create a swarmplot using a melted dataframe of all your numeric variables. The x-axis should be the variable name, the y-axis the measure. Add another dimension using color to represent species. Document takeaways from this visualization.

Ask a specific question of the data, such as: is the sepal area signficantly different in virginica compared to setosa? Answer the question through both a plot and using a mann-whitney or t-test. If you use a t-test, be sure assumptions are met (independence, normality, equal variance).

Document takeaways and any actions.

In [ ]:
train_iris.describe(include='all').T

In [ ]:
train_iris_pw_virginica = train_iris.loc[train_iris['species']== 'virginica']['petal_width']

In [ ]:
train_iris_pw_versicolor = train_iris.loc[train_iris['species']== 'versicolor']['petal_width']

In [ ]:
train_iris_pw_setosa = train_iris.loc[train_iris['species']== 'setosa']['petal_width']

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df_iris['sepal_length'], df_iris['sepal_width'], c=df_iris['species'].map({'setosa': 'pink', 'versicolor': 'purple', 'virginica': 'orange'}))
plt.xlabel('Sepal Length')
plt.ylabel('Sepal Width')
plt.title('Scatter Plot of Sepal Length vs Sepal Width')
plt.legend(df_iris['species'])
plt.show() 


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='sepal_length', y='sepal_width', hue='species', color= 'plasma' data=df_iris)
plt.xlabel('Sepal Length')
plt.ylabel('Sepal Width')
plt.title('Scatter Plot of Sepal Length vs Sepal Width')
plt.legend()
plt.show()

In [ ]:
plt.figure()
sns.pairplot(data=train_iris, hue='species', corner=True)

In [ ]:
sns.catplot(x="petal_length", y="petal_width", data=train_iris, hue="species")

In [ ]:
melted = train_iris.melt(id_vars=['species'],
               var_name='measures',
               value_name='values')
plt.title('Difference in Measurements across Species')
sns.swarmplot(x='measures',
             y='values',
             data=melted,
             hue='species'
             )
plt.xticks(rotation=45)
plt.show

## Part II
### Explore your titanic dataset more completely.

Determine drivers of the target variable
Determine if certain columns should be dropped
Determine if it would be valuable to bin some numeric columns
Determine if it would be valuable to combine multiple columns into one.
Does it make sense to combine any features?

Do you find any surprises?

Document any and all findings and takeaways in your notebook using markdown.

In [3]:
# acquire data using funciton from acquire module
df_titanic = acquire.get_titanic_data()

# peek into data of dataframe
df_titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [4]:

# clean data using funciton form prepare module
tidy_titanic = prepare.prep_titanic(titanic)

NameError: name 'prepare' is not defined